# Flight Price Prediction-Project:

## Import Libraries:


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

## Importing Dataset:

- Since data is in form of excel file we have to use pandas read_excel to load the data.

- After loading it is important to check the complete information of data as it can indication many of the hidden infomation    such as null values in a column or a row.

- Check whether any null values are there or not. if it is present then following can be done,

     1. Imputing data using Imputation method in sklearn
     2.Filling NaN values with mean, median and mode using fillna() method
     
- Describe data --> which can give statistical analysis.

In [6]:
train_data = pd.read_excel(r"C:\projects\flight_fare_project\data\Data_Train.xlsx")

In [7]:
# to display all the columns in dataset..

pd.set_option('display.max_columns',None)

In [8]:
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [9]:
# let's check the information of the data..

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [11]:
# findout the null values..

train_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [12]:
# let's drop the null values..

train_data.dropna(inplace=True)

In [14]:
train_data.isnull().sum().sum()

0

In [15]:
# findout the shape of the dataset..

train_data.shape

(10682, 11)

## EDA:

- From information of data we can see that Date_of_Journey is a object data type.
  Therefore, we have to convert this datatype into timestamp so as to use this column properly for prediction

- For this we require pandas to_datetime to convert object data type to datetime dtype.

    1:- .dt.day method will extract only day of that date
    
    2:- .dt.month method will extract only month of that date
 

- Here I am not using " .dt.year " method, because almost all the information is from the same year.

In [17]:
# let's convert the 'Date_of_journey' to date_time format..

train_data['journey_day'] = pd.to_datetime(train_data.Date_of_Journey, format="%d/%m/%Y").dt.day

In [18]:
train_data['journey_month'] = pd.to_datetime(train_data['Date_of_Journey'], format="%d/%m/%Y").dt.month

In [19]:
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9,6
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3


In [20]:
# since we converted 'Date_of_Journey' column into int type, now we can drop it as of no use..

train_data.drop(['Date_of_Journey'], axis=1, inplace=True)

In [25]:
# Departure time is leaving time of plane..
# Now we use the same method for 'Dep_time' as 'Date_of_Journey to extract the hours and minutes.. 

train_data['dep_hour'] = pd.to_datetime(train_data['Dep_Time']).dt.hour

train_data['dep_minutes'] = pd.to_datetime(train_data.Dep_Time).dt.minute

train_data.drop(['Dep_Time'], axis=1, inplace=True)

In [26]:
train_data.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month,dep_hour,dep_minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662,1,5,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25 10 Jun,19h,2 stops,No info,13882,9,6,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,23:30,5h 25m,1 stop,No info,6218,12,5,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,21:35,4h 45m,1 stop,No info,13302,1,3,16,50


In [ ]:
# Time taken by plane to reach Destination is called Duration, it is the difference between Departure time and Arrival time..

# Assigning and converting Duration column to list..
duration = list(train_data['Duration'])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # check if duration contains only hour or minutes..
        if 'h' in duration[i]:
            duration[i] = duration[i].strip() + '0m'   # adds 0min..
        else:
            duration[i] = '0h' + duration[i]        # adds 0hour..
            
duration_hours = []
duration_mins = []

for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = 'h')[0]))
    duration_mins.append(int(duration[i].split(sep = 'm')[0]))
    
